## Imports and settings

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import json
from pprint import pprint
import collections

import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
import ipywidgets as widgets
from matplotlib import gridspec
from scipy.stats import gaussian_kde
# from tqdm import tqdm

project_root = '..'
sys.path.append(project_root)

from sleeprnn.data.loader import load_dataset
from sleeprnn.helpers.reader import RefactorUnpickler
from sleeprnn.data import utils, stamp_correction
from sleeprnn.detection.feeder_dataset import FeederDataset
from sleeprnn.detection import metrics
from sleeprnn.detection.postprocessor import PostProcessor
from sleeprnn.common import constants, pkeys

SEED_LIST = [123, 234, 345, 456]
RESULTS_PATH = os.path.join(project_root, 'results')
COMPARISON_PATH = os.path.join(project_root, 'resources', 'comparison_data')

CUSTOM_COLOR = {'red': '#c62828', 'grey': '#455a64', 'blue': '#0277bd', 'green': '#43a047'} 

%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/home/ntapia/miniconda3/envs/tf/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


## Load database and predicted probabilities

In [56]:
DPI = 200

optimal_thr_for_ckpt_dict = {
    os.path.join('20190927_out_proba_cwt_grid_n2_train_mass_ss', 'p_0.5_lr_0.0001'): [0.36, 0.42, 0.62, 0.4],
    os.path.join('20191003_loss_grid_cwt_n2_train_mass_ss', 'v19_p_0.5_focal_loss_gamma_1.0'): [0.42, 0.60, 0.58, 0.44],
    os.path.join('20191003_loss_grid_cwt_n2_train_mass_ss', 'v19_p_0.5_focal_loss_gamma_1.5'): [0.44, 0.50, 0.52, 0.52],
    os.path.join('20191003_loss_grid_cwt_n2_train_mass_ss', 'v19_p_0.5_focal_loss_gamma_2.0'): [0.44, 0.52, 0.54, 0.50],
    os.path.join('20191009_focal_loss_grid_n2_train_mass_ss', 'v19_p_0.5_focal_loss_gamma_2.5'): [0.46, 0.54, 0.54, 0.52],
    os.path.join('20191009_focal_loss_grid_n2_train_mass_ss', 'v19_p_0.5_focal_loss_gamma_3.0'): [0.46, 0.52, 0.50, 0.46],
    os.path.join('20191009_focal_loss_grid_n2_train_mass_ss', 'v19_p_0.5_focal_loss_gamma_3.5'): [0.44, 0.54, 0.50, 0.50],
    os.path.join('20191009_focal_loss_grid_n2_train_mass_ss', 'v19_p_0.5_focal_loss_gamma_4.0'): [0.46, 0.52, 0.52, 0.50],
    os.path.join('20190917_out_proba_init_equal_n2_train_mass_ss', 'v11'): [0.28, 0.58, 0.56, 0.4],
    os.path.join('20190927_loss_grid_n2_train_mass_ss', 'v11_p_0.5_dice_loss_gamma_None'): [0.50, 0.50, 0.50, 0.50],
    os.path.join('20190927_loss_grid_n2_train_mass_ss', 'v11_p_0.5_focal_loss_gamma_1.0'): [0.48, 0.50, 0.54, 0.50],
    os.path.join('20190927_loss_grid_n2_train_mass_ss', 'v11_p_0.5_focal_loss_gamma_1.5'): [0.42, 0.50, 0.56, 0.44],
    os.path.join('20190927_loss_grid_n2_train_mass_ss', 'v11_p_0.5_focal_loss_gamma_2.0'): [0.46, 0.54, 0.54, 0.46],
    os.path.join('20191009_focal_loss_grid_n2_train_mass_ss', 'v11_p_0.5_focal_loss_gamma_2.5'): [0.44, 0.52, 0.46, 0.44],
    os.path.join('20191009_focal_loss_grid_n2_train_mass_ss', 'v11_p_0.5_focal_loss_gamma_3.0'): [0.48, 0.54, 0.54, 0.50],
    os.path.join('20191009_focal_loss_grid_n2_train_mass_ss', 'v11_p_0.5_focal_loss_gamma_3.5'): [0.48, 0.52, 0.52, 0.50],
    os.path.join('20191009_focal_loss_grid_n2_train_mass_ss', 'v11_p_0.5_focal_loss_gamma_4.0'): [0.46, 0.48, 0.52, 0.48]
}

names_dict = {
    os.path.join('20190927_out_proba_cwt_grid_n2_train_mass_ss', 'p_0.5_lr_0.0001'): "CWT model with cross-entropy loss",
    os.path.join('20191003_loss_grid_cwt_n2_train_mass_ss', 'v19_p_0.5_focal_loss_gamma_1.0'): "CWT model with focal loss ($\gamma=1.0$)",
    os.path.join('20191003_loss_grid_cwt_n2_train_mass_ss', 'v19_p_0.5_focal_loss_gamma_1.5'): "CWT model with focal loss ($\gamma=1.5$)",
    os.path.join('20191003_loss_grid_cwt_n2_train_mass_ss', 'v19_p_0.5_focal_loss_gamma_2.0'): "CWT model with focal loss ($\gamma=2.0$)",
    os.path.join('20191009_focal_loss_grid_n2_train_mass_ss', 'v19_p_0.5_focal_loss_gamma_2.5'): "CWT model with focal loss ($\gamma=2.5$)",
    os.path.join('20191009_focal_loss_grid_n2_train_mass_ss', 'v19_p_0.5_focal_loss_gamma_3.0'): "CWT model with focal loss ($\gamma=3.0$)",
    os.path.join('20191009_focal_loss_grid_n2_train_mass_ss', 'v19_p_0.5_focal_loss_gamma_3.5'): "CWT model with focal loss ($\gamma=3.5$)",
    os.path.join('20191009_focal_loss_grid_n2_train_mass_ss', 'v19_p_0.5_focal_loss_gamma_4.0'): "CWT model with focal loss ($\gamma=4.0$)",
    os.path.join('20190927_loss_grid_n2_train_mass_ss', 'v11_p_0.5_dice_loss_gamma_None'): "Time model with DICE loss",
    os.path.join('20190917_out_proba_init_equal_n2_train_mass_ss', 'v11'): "Time model with cross-entropy loss",
    os.path.join('20190927_loss_grid_n2_train_mass_ss', 'v11_p_0.5_focal_loss_gamma_1.0'): "Time model with focal loss ($\gamma=1.0$)",
    os.path.join('20190927_loss_grid_n2_train_mass_ss', 'v11_p_0.5_focal_loss_gamma_1.5'): "Time model with focal loss ($\gamma=1.5$)",
    os.path.join('20190927_loss_grid_n2_train_mass_ss', 'v11_p_0.5_focal_loss_gamma_2.0'): "Time model with focal loss ($\gamma=2.0$)",
    os.path.join('20191009_focal_loss_grid_n2_train_mass_ss', 'v11_p_0.5_focal_loss_gamma_2.5'): "Time model with focal loss ($\gamma=2.5$)",
    os.path.join('20191009_focal_loss_grid_n2_train_mass_ss', 'v11_p_0.5_focal_loss_gamma_3.0'): "Time model with focal loss ($\gamma=3.0$)",
    os.path.join('20191009_focal_loss_grid_n2_train_mass_ss', 'v11_p_0.5_focal_loss_gamma_3.5'): "Time model with focal loss ($\gamma=3.5$)",
    os.path.join('20191009_focal_loss_grid_n2_train_mass_ss', 'v11_p_0.5_focal_loss_gamma_4.0'): "Time model with focal loss ($\gamma=4.0$)",
}

short_names_dict = {
    os.path.join('20190927_out_proba_cwt_grid_n2_train_mass_ss', 'p_0.5_lr_0.0001'): "cwt_focal_0.0",
    os.path.join('20191003_loss_grid_cwt_n2_train_mass_ss', 'v19_p_0.5_focal_loss_gamma_1.0'): "cwt_focal_1.0",
    os.path.join('20191003_loss_grid_cwt_n2_train_mass_ss', 'v19_p_0.5_focal_loss_gamma_1.5'): "cwt_focal_1.5",
    os.path.join('20191003_loss_grid_cwt_n2_train_mass_ss', 'v19_p_0.5_focal_loss_gamma_2.0'): "cwt_focal_2.0",
    os.path.join('20191009_focal_loss_grid_n2_train_mass_ss', 'v19_p_0.5_focal_loss_gamma_2.5'): "cwt_focal_2.5",
    os.path.join('20191009_focal_loss_grid_n2_train_mass_ss', 'v19_p_0.5_focal_loss_gamma_3.0'): "cwt_focal_3.0",
    os.path.join('20191009_focal_loss_grid_n2_train_mass_ss', 'v19_p_0.5_focal_loss_gamma_3.5'): "cwt_focal_3.5",
    os.path.join('20191009_focal_loss_grid_n2_train_mass_ss', 'v19_p_0.5_focal_loss_gamma_4.0'): "cwt_focal_4.0",
    os.path.join('20190927_loss_grid_n2_train_mass_ss', 'v11_p_0.5_dice_loss_gamma_None'): "time_dice",
    os.path.join('20190917_out_proba_init_equal_n2_train_mass_ss', 'v11'): "time_focal_0.0",
    os.path.join('20190927_loss_grid_n2_train_mass_ss', 'v11_p_0.5_focal_loss_gamma_1.0'): "time_focal_1.0",
    os.path.join('20190927_loss_grid_n2_train_mass_ss', 'v11_p_0.5_focal_loss_gamma_1.5'): "time_focal_1.5",
    os.path.join('20190927_loss_grid_n2_train_mass_ss', 'v11_p_0.5_focal_loss_gamma_2.0'): "time_focal_2.0",
    os.path.join('20191009_focal_loss_grid_n2_train_mass_ss', 'v11_p_0.5_focal_loss_gamma_2.5'): "time_focal_2.5",
    os.path.join('20191009_focal_loss_grid_n2_train_mass_ss', 'v11_p_0.5_focal_loss_gamma_3.0'): "time_focal_3.0",
    os.path.join('20191009_focal_loss_grid_n2_train_mass_ss', 'v11_p_0.5_focal_loss_gamma_3.5'): "time_focal_3.5",
    os.path.join('20191009_focal_loss_grid_n2_train_mass_ss', 'v11_p_0.5_focal_loss_gamma_4.0'): "time_focal_4.0",
}

new_split_version = True
task_mode = constants.N2_RECORD
dataset_name = constants.MASS_SS_NAME
seed_id_list = [0, 1, 2, 3]
# seed_id_list = [1]

n_seeds = len(seed_id_list)
set_list = [constants.TRAIN_SUBSET, constants.VAL_SUBSET, constants.TEST_SUBSET]
# set_list = [constants.TEST_SUBSET]
which_expert = 1
verbose = False

# Load data
dataset = load_dataset(dataset_name, params={pkeys.NORM_COMPUTATION_MODE: constants.NORM_GLOBAL})
fs = dataset.fs
all_train_ids = dataset.train_ids
test_ids = dataset.test_ids

for ckpt_folder in optimal_thr_for_ckpt_dict.keys():
    print('\nProcessing %s\n' % ckpt_folder)
    
    # ckpt_folder = os.path.join('20190927_loss_grid_n2_train_mass_ss', 'v11_p_0.5_dice_loss_gamma_None')
    optimal_thr_list = optimal_thr_for_ckpt_dict[ckpt_folder]
    predictions_dict = {}
    for k in seed_id_list:
        # Restore predictions
        ckpt_path = os.path.abspath(os.path.join(
            RESULTS_PATH,
            'predictions_%s' % dataset_name,
            ckpt_folder,
            'seed%d' % k
        ))
        this_dict = {}
        for set_name in set_list:
            filename = os.path.join(
                    ckpt_path,
                    'prediction_%s_%s.pkl' % (task_mode, set_name))
            with open(filename, 'rb') as handle:
                this_pred = RefactorUnpickler(handle).load()
            this_dict[set_name] = this_pred
        predictions_dict[k] = this_dict
        print('Loaded seed %d/%d from %s' % (k + 1, n_seeds, ckpt_path))
    print('Optimal thr:', optimal_thr_list)


    # ----------------------------------------------------------------
    # ----------------------------------------------------------------
    # ----------------------------------------------------------------
    # PR test

    res_thr = 0.02
    start_thr = 0.1
    end_thr = 0.9
    n_thr = int(np.round((end_thr - start_thr) / res_thr + 1))
    thr_list = np.array([start_thr + res_thr * i for i in range(n_thr)])

    iou_thr = 0.3
    pr_curve = {}
    # Prepare expert labels
    data_test = FeederDataset(
        dataset, test_ids, task_mode, which_expert=which_expert)
    this_events = data_test.get_stamps()
    for k in seed_id_list:
        print('Processing seed %d' % k, flush=True)
        # Columns are [x: recall, y: precision]
        pr_curve[k] = np.zeros((n_thr, 2))
        for i, thr in enumerate(thr_list):
            # Prepare model predictions
            prediction_test = predictions_dict[k][constants.TEST_SUBSET]
            prediction_test.set_probability_threshold(thr)
            this_detections = prediction_test.get_stamps()

            this_stats = [metrics.by_event_confusion(this_y, this_y_pred, iou_thr=iou_thr) 
                        for (this_y, this_y_pred) in zip(this_events, this_detections)]
            this_recall = np.mean([m[constants.RECALL] for m in this_stats])
            this_precision = np.mean([m[constants.PRECISION] for m in this_stats])
            pr_curve[k][i, 0] = this_recall
            pr_curve[k][i, 1] = this_precision

    print('Done', flush=True)

    # Plot 
    bsf_name = 'Proposed (%s)' % task_mode.upper()
    fig, ax = plt.subplots(1, 1, figsize=(4, 4), dpi=DPI)
    markersize = 6
    alpha = 0.4
    text_space = 0.01
    compare_chambon = False
    show_seeds = True
    axis_lims = [0.5, 1.0]
    show_half_thr = False
    color_list = {'model_mean': CUSTOM_COLOR['blue'] , 'dosed': CUSTOM_COLOR['red']}

    # F1 score levels
    delta = 0.01 
    x_ = np.arange(1, 100) * delta 
    y_ = np.arange(1, 100) * delta 
    X, Y = np.meshgrid(x_, y_)
    Z = 2 * X * Y / (X + Y)

    levels = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    levels = [level for level in levels if (level>axis_lims[0] and level<axis_lims[1])]

    CS = ax.contour(X, Y, Z, colors='k', alpha=0.3, levels=levels)
    ax.clabel(CS, fontsize=7.5, fmt='%1.2f')

    # Chambon
    if compare_chambon and (dataset_name == constants.MASS_SS_NAME):
        dosed_rec_prec_wn = np.loadtxt(
            os.path.join(COMPARISON_PATH, 'dosed_paper', 'mass_ss_wn_pr_dosed_separately.csv'), delimiter=',')
        ax.plot(dosed_rec_prec_wn[0], dosed_rec_prec_wn[1], 
               markersize=8, c=color_list['dosed'], zorder=10, 
                label='DOSED (WN) (Paper)', marker='o', linestyle="None")
    if compare_chambon and (dataset_name == constants.MASS_KC_NAME):
        dosed_rec_prec_wn = np.loadtxt(
            os.path.join(COMPARISON_PATH, 'dosed_paper', 'mass_kc_wn_pr_dosed_separately.csv'), delimiter=',')
        ax.plot(dosed_rec_prec_wn[0], dosed_rec_prec_wn[1], 
               markersize=8, c=color_list['dosed'], zorder=10, 
                label='DOSED (WN) (Paper)', marker='o', linestyle="None")

    if show_seeds:
        # Show single seeds
        seed_shown = False
        for k in seed_id_list:
            if not seed_shown:
                seed_shown=True
                label = '%s (Seed curve)' % bsf_name
                if show_half_thr:
                    label_2 = '%s (Seed 0.5 O.P.)' % bsf_name
                else:
                    label_2 = '%s (Seed O.P.)' % bsf_name
            else:
                label = None
                label_2 = None
            ax.plot(pr_curve[k][:, 0], pr_curve[k][:, 1], 
                label=label,
                linewidth=1, color=color_list['model_mean'], zorder=7, alpha=alpha)

            if show_half_thr:
                print('Showing threshold 0.5 for all seeds')
                chosen_thr_idx = np.abs(np.asarray(thr_list) - 0.5).argmin()
                # chosen_thr_idx = np.where(np.isclose(thr_list, 0.5))[0].item()
            else:
                chosen_thr_idx = np.abs(np.asarray(thr_list) - optimal_thr_list[k]).argmin()
                # chosen_thr_idx = np.where(np.isclose(thr_list, optimal_thr_list[k]))[0].item()
            ax.scatter(pr_curve[k][chosen_thr_idx, 0], pr_curve[k][chosen_thr_idx, 1], 
                       s=50, c=color_list['model_mean'], zorder=7, alpha=alpha, label=label_2)

    ax.set_title('PR curve at IoU$>$%1.1f (%s)\n%s' % (iou_thr, dataset_name.upper(), names_dict[ckpt_folder]), fontsize=10)
    # ax.set_title('Test PR Curve with IoU$>$%1.1f (%s)' % (iou_thr, dataset_name.upper()), fontsize=10)
    ax.set_xlabel('Recall', fontsize=8)
    ax.set_ylabel('Precision', fontsize=8)
    ax.set_xlim(axis_lims)
    ax.set_ylim(axis_lims)

    lg = ax.legend(loc='lower left', labelspacing=1, fontsize=8)
    for lh in lg.legendHandles:
        lh.set_alpha(1.0)

    ax.tick_params(labelsize=8)
    ax.grid()

    fig.savefig('pr_curve_%s.png' % short_names_dict[ckpt_folder], dpi=DPI)

    plt.close(fig)

    # ----------------------------------------------------------------
    # ----------------------------------------------------------------
    # ----------------------------------------------------------------
    # Separability 

    seed_to_show = 2
    set_name = 'val'
    subject_id = 7

    # -----
    # print(ckpt_folder)

    if dataset_name in [constants.MASS_SS_NAME, constants.MASS_KC_NAME]:
        channel_name = 'C3-CLE'
    elif dataset_name in [constants.DREAMS_SS_NAME, constants.DREAMS_KC_NAME]:
        channel_name = 'Cz-A1'
    else:
        channel_name = 'F4-C4'

    pages_subset = constants.N2_RECORD

    if new_split_version:
        train_ids, val_ids = utils.split_ids_list_v2(
            all_train_ids, split_id=seed_to_show)
    else:
        train_ids, val_ids = utils.split_ids_list(
            all_train_ids, seed=SEED_LIST[seed_to_show])

    idx_dict = {'train': train_ids, 'val': val_ids, 'test': test_ids, 'alltrain': all_train_ids}
    idx_subject = idx_dict[set_name].index(subject_id)

    this_pages = dataset.get_subject_pages(subject_id=subject_id, pages_subset=pages_subset)
    this_signal = dataset.get_subject_signal(subject_id=subject_id, normalization_mode=task_mode, which_expert=which_expert)
    this_stamps = dataset.get_subject_stamps(subject_id=subject_id, pages_subset=pages_subset, which_expert=which_expert)
    this_stamps_full = this_stamps.copy()
    event_name = dataset.event_name

    this_thr = optimal_thr_list[seed_to_show]
    smaller_proba_for_fp = this_thr / 2
    prediction_set = predictions_dict[seed_to_show][set_name]
    down_factor = prediction_set.params[pkeys.TOTAL_DOWNSAMPLING_FACTOR]
    predicted_proba = prediction_set.get_subject_probabilities(subject_id)
    real_class_binary = utils.stamp2seq(this_stamps_full, 0, len(this_signal)-1)[::down_factor][:len(predicted_proba)]

    # Look for probability in real events
    mean_proba_real_list = []
    max_proba_real_list = []
    for single_stamp in this_stamps_full:
        proba_segment_idx = single_stamp // down_factor
        proba_segment = predicted_proba[proba_segment_idx[0]:proba_segment_idx[1]+1]
        mean_proba_real_list.append(np.mean(proba_segment))
        max_proba_real_list.append(np.max(proba_segment))

    # Look for probability in samples
    positive_sample_proba_list = predicted_proba[real_class_binary==1]
    negative_sample_proba_list = predicted_proba[real_class_binary==0]

    proba_bins = np.linspace(0.0, 1.0, num=21, endpoint=True) 
    fig, ax = plt.subplots(1, 2, figsize=(7, 5), dpi=DPI, sharey=True, sharex=True)

    fig.suptitle('Model output distribution (%s, S%02d)\n%s' % (dataset_name.upper(), subject_id, names_dict[ckpt_folder]), fontsize=10)

    ax[0].hist(max_proba_real_list, bins=proba_bins, color=CUSTOM_COLOR['blue'], label='Max output inside expert marks', density=True, alpha=0.6)
    ax[0].tick_params(labelsize=8)
    ax[0].legend(fontsize=8, loc='upper center')
    ax[0].set_xlim([-0.05, 1.05])
    ax[0].set_xlabel('Model output', fontsize=8)
    ax[0].set_yticks([])
    ax[0].set_ylim([0, 20])

    ax[1].hist(positive_sample_proba_list, bins=proba_bins, color=CUSTOM_COLOR['blue'], label='Output for positive samples', density=True, alpha=0.6)
    ax[1].hist(negative_sample_proba_list, bins=proba_bins, color=CUSTOM_COLOR['red'], label='Output for negative samples', density=True, alpha=0.6)
    ax[1].tick_params(labelsize=8)
    ax[1].legend(fontsize=8, loc='upper center')
    ax[1].set_xlim([-0.05, 1.05])
    ax[1].set_xlabel('Model output', fontsize=8)
    ax[1].set_yticks([])
    ax[1].set_ylim([0, 20])

    fig.savefig('output_dist_%s.png' % short_names_dict[ckpt_folder], dpi=DPI)

    plt.close(fig)

Train size: 11. Test size: 4
Train subjects: 
 [1, 3, 5, 7, 9, 10, 11, 14, 17, 18, 19]
Test subjects: 
 [2, 6, 12, 13]
Dataset mass_ss with 15 patients.
Loading from checkpoint... Loaded
Global STD: 16.482037

Processing 20190927_out_proba_cwt_grid_n2_train_mass_ss/p_0.5_lr_0.0001

Loaded seed 1/4 from /home/ntapia/Projects/GitNico/sleep-rnn/results/predictions_mass_ss/20190927_out_proba_cwt_grid_n2_train_mass_ss/p_0.5_lr_0.0001/seed0
Loaded seed 2/4 from /home/ntapia/Projects/GitNico/sleep-rnn/results/predictions_mass_ss/20190927_out_proba_cwt_grid_n2_train_mass_ss/p_0.5_lr_0.0001/seed1
Loaded seed 3/4 from /home/ntapia/Projects/GitNico/sleep-rnn/results/predictions_mass_ss/20190927_out_proba_cwt_grid_n2_train_mass_ss/p_0.5_lr_0.0001/seed2
Loaded seed 4/4 from /home/ntapia/Projects/GitNico/sleep-rnn/results/predictions_mass_ss/20190927_out_proba_cwt_grid_n2_train_mass_ss/p_0.5_lr_0.0001/seed3
Optimal thr: [0.36, 0.42, 0.62, 0.4]
Processing seed 0
Processing seed 1
Processing seed 2
Pr


Processing 20190927_loss_grid_n2_train_mass_ss/v11_p_0.5_dice_loss_gamma_None

Loaded seed 1/4 from /home/ntapia/Projects/GitNico/sleep-rnn/results/predictions_mass_ss/20190927_loss_grid_n2_train_mass_ss/v11_p_0.5_dice_loss_gamma_None/seed0
Loaded seed 2/4 from /home/ntapia/Projects/GitNico/sleep-rnn/results/predictions_mass_ss/20190927_loss_grid_n2_train_mass_ss/v11_p_0.5_dice_loss_gamma_None/seed1
Loaded seed 3/4 from /home/ntapia/Projects/GitNico/sleep-rnn/results/predictions_mass_ss/20190927_loss_grid_n2_train_mass_ss/v11_p_0.5_dice_loss_gamma_None/seed2
Loaded seed 4/4 from /home/ntapia/Projects/GitNico/sleep-rnn/results/predictions_mass_ss/20190927_loss_grid_n2_train_mass_ss/v11_p_0.5_dice_loss_gamma_None/seed3
Optimal thr: [0.5, 0.5, 0.5, 0.5]
Processing seed 0
Processing seed 1
Processing seed 2
Processing seed 3
Done
Normalizing with Global STD of 16.482037

Processing 20190927_loss_grid_n2_train_mass_ss/v11_p_0.5_focal_loss_gamma_1.0

Loaded seed 1/4 from /home/ntapia/Projec